In [ ]:
!pip install tensorboard
!pip install -qqq accelerate==0.28.0
!pip install ultralytics
!pip install fsspec==2024.10.0
!pip install -qqq datasets
!pip install torch


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 17.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 879.0/879.0 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
!pip install -U -q PyDrive
!pip install -U -q PyDrive tqdm
!pip install tqdm

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

# Google Drive 내에서 파일 경로를 확인합니다.
!ls /content/drive/MyDrive/

# 확인된 zip 파일 경로를 설정합니다.
zip_file_path = '/content/drive/MyDrive/dataset.zip'  # 실제 zip 파일 경로로 변경
extract_to_path = '/dataset/data'  # 압축을 풀 폴더 경로 설정

import zipfile
import os
from tqdm import tqdm

# 압축을 풀 폴더가 존재하지 않으면 생성합니다.
if not os.path.exists(extract_to_path):
    os.makedirs(extract_to_path)

# zip 파일 압축 해제
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    # 파일 리스트를 얻습니다.
    file_list = zip_ref.namelist()

    # tqdm을 사용하여 진행 상황을 표시합니다.
    for file in tqdm(file_list, desc="Extracting"):
        zip_ref.extract(file, extract_to_path)

print('Zip file extracted successfully.')

Mounted at /content/drive
'Colab Notebooks'   dataset   dataset.zip   tensorboard_logs


Extracting: 100%|██████████| 9363/9363 [05:43<00:00, 27.25it/s]  

Zip file extracted successfully.


In [ ]:
import os
import random
import numpy as np
import torch
import psutil
from ultralytics import YOLO
from torch.utils.tensorboard import SummaryWriter
import pandas as pd
import matplotlib.pyplot as plt
import time

# 시드 설정 함수 정의
def set_seed(seed_value=42):
    np.random.seed(seed_value)
    torch.manual_seed(seed_value)
    torch.cuda.manual_seed(seed_value)
    torch.cuda.manual_seed_all(seed_value)
    random.seed(seed_value)
    os.environ['PYTHONHASHSEED'] = str(seed_value)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

# 시드 설정
set_seed()

# CPU 코어 수 확인
cpu_count = psutil.cpu_count(logical=True)
print(f"CPU 코어 수: {cpu_count}")

# 메모리 용량 확인
memory_info = psutil.virtual_memory()
total_memory = memory_info.total / (1024 ** 3)  # GB 단위로 변환
print(f"총 메모리 용량: {total_memory:.2f} GB")

# 워커 수 자동 할당
# 메모리 사용량을 고려하여 워커 수를 설정 (여기서는 메모리의 1/8을 사용하도록 설정)
# 이 값은 필요에 따라 조정할 수 있습니다.
memory_limit = total_memory / 8  # 사용할 메모리의 1/8
worker_memory_usage = 0.5  # 워커 당 메모리 사용량 (예상 값, GB)
max_workers_by_memory = int(memory_limit / worker_memory_usage)

# CPU 코어 수와 메모리 제한 중 최소값 선택
workers = min(cpu_count * 2, max_workers_by_memory)
print(f"자동 할당된 워커 수: {workers}")

# TensorBoard 로그 디렉토리 설정
log_dir = '/content/drive/MyDrive/tensorboard_logs'
writer = SummaryWriter(log_dir)

# 모델 로드
model = YOLO("yolo11n.pt")

# 모델 학습 전 시간 기록
start_time = time.time()

# 모델 학습
results = model.train(
    data='/dataset/data/dataset.yaml',  # 데이터셋 경로
    epochs=30,  # 학습 에폭 수
    patience=10,
    imgsz=640,  # 이미지 크기
    optimizer='AdamW',  # 옵티마이저 설정
    lr0=0.001,  # 학습률 조정
    freeze=[-1],  # 동결된 레이어 확인
    workers=workers,  # 자동 할당된 데이터 로더 워커 수
    project=log_dir,  # TensorBoard 로그 디렉토리 설정
    name='train_experiment',  # 실험 이름 설정
    cache=False,  # 메모리 문제로 인해 캐시 비활성화
    batch=16,  # 배치 크기
    verbose=True,  # 상세 로그 출력
    deterministic=True,  # 재현성을 위해 결정론적 동작 설정
    seed=42,  # 시드 설정
)

# 학습 종료 후 시간 기록
end_time = time.time()
total_time = end_time - start_time
print(f"학습 시간: {total_time / 60:.2f}분")

# 학습 결과 시각화
def plot_results(log_dir):
    results_file = os.path.join(log_dir, 'train_experiment', 'results.csv')
    if os.path.exists(results_file):
        results = pd.read_csv(results_file)

        # 손실 값 시각화
        plt.figure(figsize=(10, 6))
        plt.plot(results['epoch'], results['box_loss'], label='Box Loss')
        plt.plot(results['epoch'], results['cls_loss'], label='Class Loss')
        plt.plot(results['epoch'], results['dfl_loss'], label='DFL Loss')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.title('Training Loss over Epochs')
        plt.legend()
        plt.grid(True)
        plt.show()

        # 정확도 시각화 (필요시 사용, 예: mAP)
        plt.figure(figsize=(10, 6))
        plt.plot(results['epoch'], results['mAP50'], label='mAP@0.5')
        plt.plot(results['epoch'], results['mAP50-95'], label='mAP@0.5:0.95')
        plt.xlabel('Epoch')
        plt.ylabel('mAP')
        plt.title('Validation mAP over Epochs')
        plt.legend()
        plt.grid(True)
        plt.show()
    else:
        print(f"Results file not found at {results_file}")

plot_results(log_dir)


CPU 코어 수: 2
총 메모리 용량: 12.67 GB
자동 할당된 워커 수: 3
Ultralytics 8.3.27 🚀 Python-3.10.12 torch-2.5.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=/dataset/data/dataset.yaml, epochs=30, time=None, patience=10, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=3, project=/content/drive/MyDrive/tensorboard_logs, name=train_experiment11, exist_ok=False, pretrained=True, optimizer=AdamW, verbose=True, seed=42, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=[-1], multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False,

train: Scanning /dataset/data/labels/train... 3227 images, 0 backgrounds, 12 corrupt: 100%|██████████| 3228/3228 [08:25<00:00,  6.38it/s]

train: WARNING ⚠️ /dataset/data/images/train/1_01_1_1_1_1_20210813_0000355725.jpg: ignoring corrupt image/label: broken data stream when reading image file
train: WARNING ⚠️ /dataset/data/images/train/1_02_1_1_1_1_20210719_0000120038.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /dataset/data/images/train/1_02_1_1_1_1_20210720_0000000601.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /dataset/data/images/train/1_02_1_1_1_1_20210720_0000003487.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /dataset/data/images/train/1_02_1_1_1_1_20210720_0000006949.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /dataset/data/images/train/1_02_1_1_1_1_20210720_0000007017.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /dataset/data/images/train/1_02_1_1_1_1_20210720_0000018960.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /dataset/data/images/train/1_02_1_1_1_1_20210720_0000019014.jpg: corrupt JPEG restored and saved
train: WARNING ⚠️ /dataset/data/images/

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


/usr/local/lib/python3.10/dist-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 1.4.21 (you have 1.4.20). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()
val: Scanning /dataset/data/labels/val... 1450 images, 0 backgrounds, 4 corrupt: 100%|██████████| 1450/1450 [03:47<00:00,  6.37it/s]

...

      21/30      2.43G     0.7943     0.9693      1.155         35        640: 100%|██████████| 201/201 [08:17<00:00,  2.47s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [02:40<00:00,  3.50s/it]


                   all       1446       2853       0.86      0.723      0.758      0.672

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      22/30      2.43G     0.7793     0.9124      1.149         35        640: 100%|██████████| 201/201 [07:17<00:00,  2.18s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 46/46 [02:43<00:00,  3.55s/it]


                   all       1446       2853      0.861      0.727      0.761      0.675

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      23/30      2.43G     0.7473     0.8605      1.132         29        640:  42%|████▏     | 85/201 [03:09<04:07,  2.13s/it]